In [12]:
# Импортируем библиотеки

import pandas as pd
import numpy as np
import ds_functions as dsf
import seaborn as sns
import plotly.express as px


# Читаем датасеты
tsum_exel = pd.ExcelFile('data/Data_TSUM.xlsx')
df_pars = pd.read_excel(tsum_exel, 'Data_Parsing')
df_comp = pd.read_excel(tsum_exel, 'Data_Company')

# Очищаем данные от служебных символов
rep_chars = '_|-|~|/|\\\\'
df_pars['producer_color'] = df_pars['producer_color'].str.replace(rep_chars,"", regex=True)
df_pars['producer_id'] = df_pars['producer_id'].str.replace(rep_chars,"", regex=True)

# Объединяем датафреймы
df_merged = pd.merge(
    df_pars,
    df_comp[['item_id', 'color_id', 'current price']],
    left_on='producer_id',
    right_on='item_id',
    how='left'
    ).drop(['producer_id', 'producer_color'], axis=1)

# Добавляем признак разницы цены клиента и конкурентов в процентах
df_merged['persent_diff'] = 100 - round(
    df_merged['price'] / df_merged['current price'] * 100)

# Добавляем признак Бренд/Категория для удобной визуализации и поиска аномалий. 
# Оптимизированное предложение от Chat GPT
df_merged = df_merged.assign(brand_cat=df_merged['brand'] + '/' + df_merged['Category'])

In [13]:
# Выводим среднее значение отклонения по категориям и описание столбца с разницей в цене
grouped_df = df_merged.groupby(['brand', 'Category']).mean().round()
display(grouped_df)
display(df_merged['persent_diff'].describe())

price  current price  persent_diff
brand        Category                                    
Brioni       Bags      203.0          201.0          -4.0
             Shoes     238.0          233.0          -5.0
Stone Island Bags      210.0          218.0           1.0
             Shoes     225.0          225.0          -4.0
Valentino    Bags      229.0          232.0          -3.0
             Shoes     209.0          223.0          -0.0

count    75.000000
mean     -2.626667
std      28.754682
min     -68.000000
25%     -18.000000
50%       4.000000
75%      18.500000
max      42.000000
Name: persent_diff, dtype: float64

### Search and eliminate otliers

In [14]:
#  Строим график поиска выбросов
box = px.box(
    data_frame = df_merged,
    x = 'persent_diff',
    y = 'brand_cat',
    width = 800,
    height = 400,
    color='brand_cat'
    )

box.show()

In [15]:
# Создаем список значений признака Бренд/Категория для проверки на выбросы в цикле.
brand_list = list(df_merged['brand_cat'].value_counts().index)

# Создаем список выбросов.
outliers_list_id = []

# Проверем в цикле признак Бренд/Категория на выбросы и добавляем в список выбросов.
for brand_name in brand_list:
    mask = df_merged['brand_cat'] == brand_name
    outliers, cleaned = dsf.outliers_iqr(df_merged[mask], 'persent_diff')
    print(f'Tjuki outliers for brand {brand_name}: {outliers.shape[0]}')
    if not outliers.empty:
        outliers_list_id.extend(list(outliers['item_id']))

# Очевидно что после разбивки данных по признакам бренда и категории внутри бренда 
# видны незначительные выбросы в категориях Brioni/Shoes и Stone Island/Bags, 
# определенные методом Тьюки.

# Выводим список 'item_id' выбросов.
print(outliers_list_id)

# Добавляем признак 'outliner' в таблицу.
for i in outliers_list_id:
    df_merged.loc[df_merged['item_id'] == i, 'outlier'] = 'Yes'

# Делаем экспорт итоговой таблицы в Exel.
df_merged.to_excel('data/Data_TSUM_modified.xlsx')

Tjuki outliers for brand Valentino/Shoes: 0
Tjuki outliers for brand Brioni/Shoes: 4
Tjuki outliers for brand Stone Island/Shoes: 0
Tjuki outliers for brand Valentino/Bags: 0
Tjuki outliers for brand Brioni/Bags: 0
Tjuki outliers for brand Stone Island/Bags: 1
['bbbbb111116', 'bbbbb111117', 'bbbbb111120', 'bbbbb111124', 'sssss111126']


In [16]:
# Chat GPT optimization


import concurrent.futures

# Создаем список значений признака Бренд/Категория для проверки на выбросы в цикле.
brand_list = list(df_merged['brand_cat'].value_counts().index)

# Create a boolean mask that filters for all the brands in brand_list
mask = df_merged['brand_cat'].isin(brand_list)

# Create a list of outliers
outliers_list_id = []

def compute_outliers(brand_name):
    mask_brand = (df_merged['brand_cat'] == brand_name) & mask
    outliers, cleaned = dsf.outliers_iqr(df_merged[mask_brand], 'persent_diff')
    print(f'Tjuki outliers for brand {brand_name}: {outliers.shape[0]}')
    if not outliers.empty:
        outliers_list_id.extend(list(outliers['item_id']))

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(compute_outliers, brand_list)

# Set the 'outlier' column to 'Yes' for rows with 'item_id' in outliers_list_id
mask_outliers = df_merged['item_id'].isin(outliers_list_id)
df_merged.loc[mask_outliers, 'outlier'] = 'Yes'

# Export the modified dataframe to an Excel file
df_merged.to_excel('data/Data_TSUM_modified.xlsx')